<a href="https://colab.research.google.com/github/nadaaym/TChecker/blob/main/TChecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf
tf.__version__
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 114.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.4 MB/s eta 0:00:00


In [ ]:
!pip install pytorch-ignite

In [ ]:


import numpy as np
import pandas as pd
from transformers import BertTokenizer,BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pack_padded_sequence
from torch.optim import AdamW
from tqdm import tqdm
from argparse import ArgumentParser
from ignite.engine import Engine,State, Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.handlers import EarlyStopping
from ignite.contrib.handlers import TensorboardLogger, ProgressBar
from ignite.utils import convert_tensor
from torch.optim.lr_scheduler import ExponentialLR
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip3 install emoji==0.6.0

In [ ]:
from transformers import BertweetTokenizer
def Bert_Tokenizer(model_name):
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    tokenizer = BertweetTokenizer.from_pretrained(model_name)
    return tokenizer
tokenizer = Bert_Tokenizer('vinai/bertweet-base')

In [ ]:
import pandas as pd
import numpy as np
dir="./drive/My Drive/Colab Notebooks/user_features/"
train=pd.read_csv(dir+"gossip_train_users.csv")   #sampled data, such that the two classes are balanced
val=pd.read_csv(dir+"gossip_val_users.csv")
test=pd.read_csv(dir+"gossip_test_users.csv")


In [ ]:
train['target']=train['target'].astype('category')
train['target']=train['target'].cat.codes.astype('float')  #fake:0, real:1

In [ ]:
val['target']=val['target'].astype('category')
val['target']=val['target'].cat.codes.astype('float')  #fake:0, real:1

In [ ]:
test['target']=test['target'].astype('category')
test['target']=test['target'].cat.codes.astype('float')  #fake:0, real:1

In [ ]:
import re
class TextDataset(Dataset):
    def __init__(self,df,tokenizer,twtokenizer,max_len,tw_len):

        self.bert_encode = tokenizer
        self.bertweet_encode=twtokenizer
        self.texts = df.tweet_text.values
        self.content=df.news_content.values
        self.labels = df.target.values
        self.max_len = max_len
        self.tw_len=tw_len
        self.df=df

    def __len__(self):

        return len(self.texts)


    def __getitem__(self,idx):

        tw_tokens,tw_mask,tw_tokens_len = self.get_tw_mask(self.texts[idx],self.tw_len)
        co_tokens,co_mask,co_tokens_len=self.get_token_mask(self.content[idx],self.max_len)
        label = self.labels[idx]

        return [torch.tensor(tw_tokens),torch.tensor(tw_mask),torch.tensor(tw_tokens_len),torch.tensor(co_tokens),torch.tensor(co_mask),torch.tensor(co_tokens_len)],label

    def get_token_mask(self,text,max_len):


        tokens = []
        mask = []
        text = self.bert_encode.encode(text)
        size = len(text)
        if size < max_len:
          pads = self.bert_encode.encode(['PAD']*(max(0,max_len-size)))

          tokens[:max(max_len,size)] = text[:max(max_len,size)]
          tokens = tokens + pads[1:-1]
          mask = [1]*size+[0]*len(pads[1:-1])
        elif size >= max_len:
          tokens[:max_len] = text[:max_len]
          mask = [1]*max_len

        tokens_len = len(tokens)

        return tokens,mask,tokens_len

    def get_tw_mask(self,text,tw_len):

        tokens = []
        mask = []
        text=re.sub(r'http\S+', '', text)
        text = self.bertweet_encode.encode(text)
        size = len(text)
        if size < tw_len:
          pads = self.bertweet_encode.encode(['PAD']*(max(0,tw_len-size)))

          tokens[:max(tw_len,size)] = text[:max(tw_len,size)]
          tokens = tokens + pads[1:-1]
          mask = [1]*size+[0]*len(pads[1:-1])
        elif size >= tw_len:
          tokens[:tw_len] = text[:tw_len]
          mask = [1]*tw_len

        tokens_len = len(tokens)


        return tokens,mask,tokens_len

In [ ]:
def get_data_loaders():
    train_dataset = TextDataset(train,tokenizer=tokenizer,twtokenizer=twtokenizer ,max_len=512,tw_len=128)
    train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=16,shuffle=True)
    valid_dataset = TextDataset(val,tokenizer=tokenizer,twtokenizer=twtokenizer ,max_len=512,tw_len=128)
    valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=16,shuffle=True)

    return train_loader , valid_loader

In [ ]:
from transformers import BertConfig, AutoModel


class TChecker(nn.Module):
    def __init__(self):
        super().__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.twbert=AutoModel.from_pretrained('vinai/bertweet-base')
        self.hidden_size = self.bert.config.hidden_size
        self.hidden_size_tw=self.twbert.config.hidden_size
        self.LSTM = nn.LSTM(self.hidden_size,self.hidden_size,bidirectional=True) #self.hidden_size+7
        self.twLSTM=nn.LSTM(self.hidden_size_tw,self.hidden_size_tw,bidirectional=True)
        self.clf = nn.Linear((self.hidden_size*4),1)


    def forward(self,inputs):

        tw_encoded_layers, tw_pooled_output = self.twbert(input_ids=inputs[0],attention_mask=inputs[1],return_dict = False)
        ti_encoded_layers,ti_pooled_output=self.bert(inputs[3],attention_mask=inputs[4],return_dict=False)

        tw_encoded_layers = tw_encoded_layers.permute(1,0,2)
        ti_encoded_layers=ti_encoded_layers.permute(1,0,2)
        =
        tw_enc_hiddens, (tw_last_hidden, tw_last_cell) = self.twLSTM(pack_padded_sequence(tw_encoded_layers,inputs[2].cpu()))
        ti_enc_hiddens,(ti_last_hidden,ti_last_cell)=self.LSTM(pack_padded_sequence(ti_encoded_layers,inputs[5].cpu()))
        output_hidden = torch.cat((tw_last_hidden[0], tw_last_hidden[1],ti_last_hidden[0],ti_last_hidden[1]), dim=1)
        output_hidden = F.dropout(output_hidden,0.2)
        output = self.clf(output_hidden)

        return F.sigmoid(output)

In [ ]:
def _prepare_batch(batch, device=None, non_blocking=False):

    x, y = batch

    return (convert_tensor(x, device=device, non_blocking=non_blocking),
            convert_tensor(y, device=device, non_blocking=non_blocking))
def create_supervised_trainer1(model, optimizer, loss_fn, metrics={}, device=None):

    def _update(engine, batch):
        model.train()
        optimizer.zero_grad()
        x, y = _prepare_batch(batch, device=device)
        y_pred = model(x)
        y=y.unsqueeze(1)
        loss = loss_fn(y_pred, y.float())
        loss.backward()
        optimizer.step()
        return loss.item(), y_pred, y

    def _metrics_transform(output):
        return output[1], output[2]

    engine = Engine(_update)

    for name, metric in metrics.items():
        metric._output_transform = _metrics_transform
        metric.attach(engine, name)

    return engine

def create_supervised_evaluator1(model, metrics=None,
                                device=None, non_blocking=False,
                                prepare_batch=_prepare_batch,
                                output_transform=lambda x, y, y_pred: (y_pred, y,)):

    metrics = metrics or {}

    if device:
        model

    def _inference(engine, batch):
        model.eval()
        with torch.no_grad():
            x, y = prepare_batch(batch, device=device, non_blocking=non_blocking)
            y_pred = model(x)
            y=y.unsqueeze(1)
            return output_transform(x, y.float(), y_pred)

    engine = Engine(_inference)

    for name, metric in metrics.items():
        metric.attach(engine, name)

    return engine

In [ ]:
def run(log_interval=100,epochs=4,lr=0.000006):
    train_loader ,valid_loader = get_data_loaders()
    model = TChecker()
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

    criterion = nn.BCELoss()
    optimizer = AdamW(model.parameters(), lr=lr)
    lr_scheduler = ExponentialLR(optimizer, gamma=0.90)
    trainer = create_supervised_trainer1(model.to(device), optimizer, criterion, device=device)
    evaluator = create_supervised_evaluator1(model.to(device), metrics={'BCELoss': Loss(criterion)}, device=device)

    if log_interval is None:
        e = Events.ITERATION_COMPLETED
        log_interval = 1
    else:
        e = Events.ITERATION_COMPLETED(every=log_interval)

    desc = "loss: {:.4f} | lr: {:.4f}"
    pbar = tqdm(
        initial=0, leave=False, total=len(train_loader),
        desc=desc.format(0, lr)
    )
    @trainer.on(e)
    def log_training_loss(engine):
        pbar.refresh()
        lr = optimizer.param_groups[0]['lr']
        pbar.desc = desc.format(engine.state.output[0], lr)
        pbar.update(log_interval)

    @trainer.on(Events.EPOCH_COMPLETED)
    def update_lr_scheduler(engine):
        lr_scheduler.step()



    @trainer.on(Events.EPOCH_COMPLETED)
    def log_training_results(engine):
        evaluator.run(train_loader)
        metrics = evaluator.state.metrics
        avg_loss = metrics['BCELoss']
        tqdm.write(
            "Train Epoch: {} BCE loss: {:.2f}".format(engine.state.epoch, avg_loss)
        )

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(engine):
        pbar.refresh()
        evaluator.run(valid_loader)
        metrics = evaluator.state.metrics
        avg_loss = metrics['BCELoss']
        tqdm.write(
            "Valid Epoch: {} BCE loss: {:.2f}".format(engine.state.epoch, avg_loss)
        )
        pbar.n = pbar.last_print_n = 0


    try:
        trainer.run(train_loader, max_epochs=epochs)

    except Exception as e:
        import traceback
        print(traceback.format_exc())
    return model

In [ ]:
model =run()

In [ ]:
class TestTextDataset(Dataset):
    def __init__(self,df,tokenizer,max_len):

        self.bert_encode = tokenizer
        self.texts = df.tweet_text.values
        self.max_len = max_len

    def __len__(self):

        return len(self.texts)

    def __getitem__(self,idx):

        tokens,mask,tokens_len = self.get_token_mask(self.texts[idx],self.max_len)
        return [torch.tensor(tokens),torch.tensor(mask),torch.tensor(tokens_len)]

    def get_token_mask(self,text,max_len):

        tokens = []
        mask = []
        text = self.bert_encode.encode(text)
        size = len(text)
        if size < max_len:
          pads = self.bert_encode.encode(['PAD']*(max(0,max_len-size)))
          #print("padded")
          tokens[:max(max_len,size)] = text[:max(max_len,size)]
          tokens = tokens + pads[1:-1]
          mask = [1]*size+[0]*len(pads[1:-1])
        elif size >= max_len:
          tokens[:max_len] = text[:max_len]
          mask = [1]*max_len

        tokens_len = len(tokens)

        return tokens,mask,tokens_len

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.eval()
predictions = []
test_dataset = TestTextDataset(test,tokenizer=tokenizer,max_len=128)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=32,shuffle=False)
with torch.no_grad():
    for idx , (inputs) in tqdm(enumerate(test_loader),total=len(test_loader)):
        inputs = [a.to(device) for a in inputs]
        preds = model(inputs)
        predictions.append(preds.cpu().detach().numpy())

predictions = np.vstack(predictions)

In [ ]:
predictions=np.round(predictions).astype(int)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(test.target,predictions))